In [1]:
# importing the libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
#import tqdm 
#from tqdm import tqdm
# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from torch import nn 
from torch.utils.data import Dataset, DataLoader
#import torchvision.transforms as transforms
from sklearn.preprocessing import OneHotEncoder
import wandb

In [4]:
301*250

75250

Splitting Dataset into chunks 
Run only once

In [5]:
root = '/scratch/proj/napamegs/all_data/'
sub_roots = os.listdir(root)
i = 0 

for sub_root in sub_roots:
    os.chdir(os.path.join(root,sub_root))
    
    for j,(suub_dir, sub_sub_dir, files) in enumerate(os.walk(os.getcwd())):
        
        for file in files:    
                path = os.path.join(suub_dir,file)
                for chunk in pd.read_csv(path, chunksize=150500):
                    chunk.to_csv(f'/scratch/proj/napamegs/all_chunks/chunk{i}{j}.csv', index=False)
                    i += 1
                print(path)

/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_120.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_40.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_105.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_15.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_30.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_130.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_60.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_55.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_50.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_75.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_10.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_80.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_150.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_100.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_140.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_5.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.5/TRS_20.csv
/scratch/proj/napamegs/all_data/ZrN/ZrN_n2.

Custom dataloder 

In [5]:
INPUT_DATA_DIR = '/scratch/proj/napamegs/all_chunks/'
TRAIN_DATA_COEFFICIENT = 0.80

files = []

for (dirpath, dirnames, filenames) in os.walk(INPUT_DATA_DIR):
    files.extend(filenames)
    break

train_files_finish = int(len(files) * TRAIN_DATA_COEFFICIENT)
train_files = files[0:train_files_finish]
test_files = files[train_files_finish:len(files)]


In [6]:
class get_data(Dataset):
    
    def __init__(self, trainTest_file, root_dir):
        self.annotations = trainTest_file
        self.root_dir = root_dir
        
    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        full_path = os.path.join(self.root_dir, self.annotations[index])
        df = pd.read_csv(full_path)
        df['met'] = df['met'].map({'Ag': 1, 'Au': 2, 'Cu':3, 'TiN':4, 'ZrN':5})

        X = df.iloc[:,-1].values.astype('float64')
       

        cat_target = df.iloc[0:1,0:4].values.astype('float64')
        num_target = df.iloc[0:1,4:7].values.astype('float64')
     
        X,cat_target,num_target=torch.FloatTensor(X),torch.FloatTensor(cat_target),torch.FloatTensor(num_target)
        return (X, cat_target,num_target)

In [7]:
in_channel = 1
num_classes = 6
batch_size = 1
num_epochs = 500

Train _test DataLoader

In [8]:
train_set = get_data(
    trainTest_file=train_files,
    root_dir= '/scratch/proj/napamegs/all_chunks/'
)

test_set = get_data(
    trainTest_file=test_files,
    root_dir= '/scratch/proj/napamegs/all_chunks/',
)

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

In [9]:
for batch_idx, (data, target1, target2) in enumerate(train_loader):
    print(f'{batch_idx} data{target1}, {target2}')
    print(target1.size(), target2.size())
    break
    

0 datatensor([[[4.0000, 0.0000, 3.7000, 1.5800]]]), tensor([[[145.,  50.,  22.]]])
torch.Size([1, 1, 4]) torch.Size([1, 1, 3])


Architecture

In [10]:
import torch.nn.functional as F

In [11]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()


        self.cnn1 = Conv2d(in_channels=1, out_channels=16, kernel_size=(5,1), stride=2,padding=2)
        self.cnn2 = Conv2d(in_channels=16, out_channels=32, kernel_size=(5,3), stride=2, padding=2)
        self.cnn3 = Conv2d(in_channels=32, out_channels=64, kernel_size=(5,3), stride=2, padding=2)
        self.cnn4 = Conv2d(in_channels=64, out_channels=128, kernel_size=(5,3), stride=2,padding= 2)
        self.cnn5 = Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), stride=2,padding=1)

        self.fc1 = nn.Linear(5120, 2600)
        self.fc2 = nn.Linear(2600, 1300)
        self.fc3 = nn.Linear(1300,512)
        self.fc4 = nn.Linear(512,256)
        self.fc5 = nn.Linear(256,64)
        self.obj1 = nn.Linear(64,3)    
        self.obj2 = nn.Linear(64,4)


    # Defining the forward pass    
    def forward(self, x):
        
        x = F.relu(self.cnn1(x))   # convolutional 1        input size [H : w: D] = [301 : 1: 1] ->  [151: 3 : 16]
        x = F.relu(self.cnn2(x))   # convolutional 2        input size [H : w: D] = [151: 3 : 16] -> [76 : 3 : 32]
        x = F.relu(self.cnn3(x))   # convolutional 3        input size [H : w: D] = [76 : 3 : 32] -> [38 : 3 : 64]
        x = F.relu(self.cnn4(x))   #convolutional 4         input size [H : w: D] = [38 : 3 : 64] -> [19: 3 : 128]
        x = F.relu(self.cnn5(x))   # convolutional 5        input size [H : w: D] = [19: 3 : 128] -> [10 : 2: 256]
        x = x.reshape(x.size(0), -1)   # Flatten  5120
        x =F.relu(self.fc1(x))    # mlp 1  [5120 -> 2600]   
        x =F.relu(self.fc2(x))   # mlp 2   [ 2600 -> 1300]
        x = F.relu(self.fc3(x))   # mlp 3   [1300 -> 512]
        x = F.relu(self.fc4(x))   # mlp 4   [512 -> 256]
        x = F.relu(self.fc5(x))   # mlp 5   [256 -> 64]
        out_obj1 = self.obj1(x)   # output for numerical features  [64 -> 3]
        out_obj2 = self.obj2(x)    # output for categorical features [64 ->4]
      
        return out_obj1,out_obj2

Checking the device for training 

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Defining the Model

In [13]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.01)
# defining the loss function
criterion1 = nn.MSELoss()
criterion2 = nn.CrossEntropyLoss()
# Get model to cuda if possible
model.to(device=device)

    
print(model)

/home/proj/napamegs/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:120: UserWarning: 
    Found GPU%d %s which is of cuda capability %d.%d.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is %d.%d.
    
  warnings.warn(old_gpu_warn.format(d, name, major, minor, min_arch // 10, min_arch % 10))


Net(
  (cnn1): Conv2d(1, 16, kernel_size=(5, 1), stride=(2, 2), padding=(2, 2))
  (cnn2): Conv2d(16, 32, kernel_size=(5, 3), stride=(2, 2), padding=(2, 2))
  (cnn3): Conv2d(32, 64, kernel_size=(5, 3), stride=(2, 2), padding=(2, 2))
  (cnn4): Conv2d(64, 128, kernel_size=(5, 3), stride=(2, 2), padding=(2, 2))
  (cnn5): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (fc1): Linear(in_features=5120, out_features=2600, bias=True)
  (fc2): Linear(in_features=2600, out_features=1300, bias=True)
  (fc3): Linear(in_features=1300, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=64, bias=True)
  (obj1): Linear(in_features=64, out_features=3, bias=True)
  (obj2): Linear(in_features=64, out_features=4, bias=True)
)


WANDB configuration 

In [ ]:
wandb.init(project="CNN inverse model", entity="bappaditya")
wandb.config = {
  "learning_rate": 0.01,
  "epochs": 500,
  "batch_size" : 301
}


Training 

In [27]:
for epoch in range(num_epochs):

    for batch_idx, (data, cat_target,num_target) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        cat_target = cat_target.to(device=device)
        num_target = num_target.to(device=device)

        data = data.reshape(1,1,301,1)
        cat_target = cat_target.reshape(1,4)
        num_target = num_target.reshape(1,3)


        # forward
        Common_block = model(data)
        loss1 = criterion1(Common_block[0],num_target)
        loss2 = criterion2(Common_block[1],cat_target)
        loss = loss1 +loss2 


        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
    #wandb.log({"loss": loss.item()})
    #wandb.watch(model)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.